In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r"D:\aegis\Advance Python\Heath Care\heart.csv")
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [3]:
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

## Model 1: Random forest

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X=df.drop('target',axis= True)
y= df.target

In [6]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=20,stratify = y)

In [7]:
from sklearn.ensemble import RandomForestClassifier

In [8]:
model_1 = RandomForestClassifier()
model_1.fit(X_train, y_train)
y_pred1 = model_1.predict(X_test)

In [9]:
from sklearn.metrics import accuracy_score,mean_squared_error

In [10]:
accuracy_score(y_test,y_pred1)

0.8032786885245902

In [11]:
from sklearn.model_selection import RandomizedSearchCV

In [12]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 110, num = 11)]
#max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

In [13]:
#Create the random grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [14]:
# Use the random grid to search for best hyperparameters

# First create the base model to tune
rf = RandomForestClassifier()

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = param_grid, 
                               n_iter = 100, cv = 3, verbose=2, random_state = 10, n_jobs = -1)

# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.1min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [1, 11, 22, 33, 44, 55, 66,
                                                      77, 88, 99, 110],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=10, verbose=2)

In [15]:
rf_random.best_params_

{'n_estimators': 1800,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 88,
 'bootstrap': False}

In [35]:
# Manually provide the best parameters to model for training
rf_reg = RandomForestClassifier(**{'n_estimators': 1800,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 88,
 'bootstrap': False})

In [36]:
rf_reg.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, max_depth=88, max_features='sqrt',
                       min_samples_leaf=2, min_samples_split=5,
                       n_estimators=1800)

In [37]:
y_pred2 = rf_reg.predict(X_test)
accuracy_score(y_test,y_pred2)

0.819672131147541

In [38]:
print("Score based on testing data:")
print("Adjusted R-squared : {}%".format((rf_reg.score(X_test,y_test)*100).round(2)))
print("RMSE :", np.sqrt(mean_squared_error(y_pred2,y_test)).round(2))

Score based on testing data:
Adjusted R-squared : 81.97%
RMSE : 0.42


## Model 2: KNN

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X=df.drop('target',axis= True)
y= df.target

In [39]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=10,stratify = y)

In [40]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [41]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('knn', KNeighborsClassifier()))
model = Pipeline(estimators)

In [42]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

kfold = StratifiedKFold(n_splits=10, shuffle = True, random_state=7)
results = cross_val_score(model, X_train, y_train, cv=kfold)

In [43]:
print(results.mean())
print(results.std())

0.8098333333333333
0.10247343829283544


In [44]:
search_space = [{
                 'knn__n_neighbors': range(1,20),
                 'knn__weights': ['distance']
                }]

In [45]:
from sklearn.model_selection import GridSearchCV

kfold = StratifiedKFold(n_splits=10, shuffle = True, random_state=10)
clf = GridSearchCV(model, search_space, cv=kfold, return_train_score=True ,verbose=False)
clf = clf.fit(X_train, y_train)

In [46]:
clf.best_estimator_

Pipeline(steps=[('standardize', StandardScaler()),
                ('knn',
                 KNeighborsClassifier(n_neighbors=19, weights='distance'))])

In [52]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('knn', KNeighborsClassifier(n_neighbors = 19)))
model = Pipeline(estimators)

In [53]:
model.fit(X_train, y_train)

Pipeline(steps=[('standardize', StandardScaler()),
                ('knn', KNeighborsClassifier(n_neighbors=19))])

In [54]:
Predicted_Class = model.predict(X_test)
print(accuracy_score(y_test,Predicted_Class))

0.8852459016393442


In [55]:
print("Score based on testing data:")
print("Adjusted R-squared : {}%".format((model.score(X_test,y_test)*100).round(2)))
print("RMSE :", np.sqrt(mean_squared_error(Predicted_Class,y_test)).round(2))

Score based on testing data:
Adjusted R-squared : 88.52%
RMSE : 0.34


In [56]:
import pickle
filename = 'Heart_model.pkl'
pickle.dump(model, open(filename, 'wb'))